In [1]:
import cv2
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
import numpy as np
import os
from PIL import Image
from torchvision import transforms

C:\Users\rcrdo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Inicializar el dispositivo: GPU si está disponible, si no, usar CPU ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Inicializar el detector de rostros MTCNN ---
# Este modelo detecta y recorta automáticamente las caras
mtcnn = MTCNN(image_size=160, margin=0, device=device)

# --- Cargar el modelo de reconocimiento facial FaceNet (entrenado con VGGFace2) ---
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [3]:
#Lista para guardar las imagenes de las personas con acceso
authorized_embeddings = []
# Lista para guardar los Nombres de las personas con acceso
authorized_names = []

In [ ]:
# Leer imágenes desde la carpeta "faces"
path = "faces"
for file in os.listdir(path):
    img_path = os.path.join(path, file)
    img = Image.open(img_path)

    # Usar MTCNN para extraer y recortar el rostro de la imagen
    face = mtcnn(img)

    if face is not None:
        # Obtener el embedding (vector de 512 elementos) del rostro recortado
        embedding = model(face.unsqueeze(0).to(device))
        print(embedding.detach().cpu())
        authorized_embeddings.append(embedding.detach().cpu())  # Guardar el vector
        authorized_names.append(os.path.splitext(file)[0])       # Guardar el nombre del archivo sin extensión


In [13]:
# Iniciar cámara
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Si no hay fotograma, salir del bucle
    
    # Convertir el fotograma de OpenCV (BGR) a formato PIL (RGB)
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Usar MTCNN para detectar y recortar el rostro
    face = mtcnn(img)

    if face is not None:
        # Obtener el embedding del rostro detectado
        embedding = model(face.unsqueeze(0).to(device))
        embedding = embedding.detach().cpu()

        # Calcular la distancia entre el rostro detectado y cada rostro autorizado
        distances = [torch.norm(e - embedding).item() for e in authorized_embeddings]
        min_dist = min(distances)  # Tomar la menor distancia
        index = distances.index(min_dist)

        # Decidir si es un rostro autorizado o no, según el umbral
        name = "Desconocido"
        if min_dist < 0.9:  # Umbral recomendado (puedes ajustarlo)
            name = authorized_names[index]

        # Mostrar el nombre y la distancia en pantalla
        cv2.putText(frame, f"{name} ({min_dist:.2f})", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0) if name != "Desconocido" else (0, 0, 255), 2)

    # Mostrar el fotograma con los resultados
    cv2.imshow("Reconocimiento Facial (PyTorch)", frame)

    # Salir al presionar la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Liberar recursos al terminar ---
cap.release()
cv2.destroyAllWindows()